# 台灣公司資料

* 抓取檔案來自[台灣公司資料](http://company.g0v.ronny.tw/)
* 欄位定義看[這邊](http://company.g0v.ronny.tw/index/columns)
* 資料下載位置在[這裡](http://ronnywang-twcompany.s3-website-ap-northeast-1.amazonaws.com/index.html)

In [1]:
import requests
import pandas as pd 

In [2]:
url = 'http://ronnywang-twcompany.s3-website-ap-northeast-1.amazonaws.com/files/'

In [3]:
# http://ronnywang-twcompany.s3-website-ap-northeast-1.amazonaws.com/files/00000000.json.gz
file_indexes = ['00000000','10000000','20000000','30000000','40000000','50000000','60000000','70000000','80000000','90000000']
# file_name = file_indexes[0] + '.json.gz'
path = '../data/'

In [4]:
url + file_indexes[0] + '.jsonl.gz'

'http://ronnywang-twcompany.s3-website-ap-northeast-1.amazonaws.com/files/00000000.jsonl.gz'

In [5]:
def download_gz(path=path,file_indexes=file_indexes):
    ## 大型公司資料 00000000.json.gz 
    for item in file_indexes:
        url_data = url + item + '.jsonl.gz'
        res = requests.get(url_data)
        storage_path = path + item + '.jsonl.gz'
        with open(storage_path,'wb') as handle:
            if not res.ok:
                print('oops, something wrong!')
            else:
                for block in res.iter_content(1024):
                    handle.write(block)
                print '{} is downloaded'.format(item+'.jsonl.gz')
    ## 小企業資料 business-00000000.json.gz
    for item in file_indexes:
        file_path = 'bussiness-' + item + '.jsonl.gz'
        url_data = url + file_path
        res = requests.get(url_data)
        storage_path = path + file_path        
        with open(storage_path,'wb') as handle:
            if not res.ok:
                print('oops, something wrong')
            else:
                for block in res.iter_content(1024):
                    handle.write(block)        
                print '{} is downloaded'.format(file_path)

In [6]:
download_gz()

00000000.jsonl.gz is downloaded
10000000.jsonl.gz is downloaded
20000000.jsonl.gz is downloaded
30000000.jsonl.gz is downloaded
40000000.jsonl.gz is downloaded
50000000.jsonl.gz is downloaded
60000000.jsonl.gz is downloaded
70000000.jsonl.gz is downloaded
80000000.jsonl.gz is downloaded
90000000.jsonl.gz is downloaded
bussiness-00000000.jsonl.gz is downloaded
bussiness-10000000.jsonl.gz is downloaded
bussiness-20000000.jsonl.gz is downloaded
bussiness-30000000.jsonl.gz is downloaded
bussiness-40000000.jsonl.gz is downloaded
bussiness-50000000.jsonl.gz is downloaded
bussiness-60000000.jsonl.gz is downloaded
bussiness-70000000.jsonl.gz is downloaded
bussiness-80000000.jsonl.gz is downloaded
bussiness-90000000.jsonl.gz is downloaded


## 處理.gz檔案

In [1]:
import gzip
import json

In [2]:
data = []
with gzip.open('../data/00000000.jsonl.gz') as f:
    for line in f:
        data.append(line)

In [3]:
data0 = json.loads(data[0])
data1 = json.loads(data[1])
data1

{u'id': u'16',
 u'\u4ee3\u8868\u4eba\u59d3\u540d': u'\u738b\u632f\u6797',
 u'\u516c\u53f8\u540d\u7a31': u'\u5bcc\u53f0\u6a5f\u68b0\u958b\u767c\u5efa\u8a2d\u6709\u9650\u516c\u53f8',
 u'\u516c\u53f8\u6240\u5728\u5730': u'\u81fa\u5317\u5e02\u677e\u5c71\u5340\u5357\u4eac\u6771\u8def4\u6bb5120\u5df719\u5f0423\u865f\ufe391\ufe3a',
 u'\u516c\u53f8\u72c0\u6cc1': u'\u6838\u51c6\u8a2d\u7acb',
 u'\u6240\u71df\u4e8b\u696d\u8cc7\u6599': [[u'',
   u'\u6a5f\u68b0\u958b\u767c\u5de5\u7a0b\u4e4b\u627f\u652c'],
  [u'',
   u'\u9053\u8def\u571f\u6728\u5efa\u7bc9\u6a4b\u6a11\u7b49\u4e4b\u5de5\u7a0b\u4e4b\u627f\u652c'],
  [u'', u'\u571f\u5730\u958b\u767c\u898f\u5283\u6539\u826f\u8a2d\u8a08'],
  [u'', u'\u793e\u5340\u570b\u6c11\u4f4f\u5b85\u7814\u7a76\u53ca\u8a2d\u8a08'],
  [u'',
   u'\u8207\u524d\u5404\u9805\u6709\u95dc\u9032\u7bc9\u6750\u6599\u7d93\u92b7\u53ca\u7522\u88fd'],
  [u'',
   u'\u6709\u95dc\u524d\u9805\u696d\u52d9\u4e4b\u7d93\u71df\u8207\u6295\u8cc7']],
 u'\u6700\u5f8c\u6838\u51c6\u8b8a\u66f4\u65e

In [4]:
print '/'.join(data0.keys())

代表人姓名/所營事業資料/經理人名單/核准設立日期/公司所在地/資本總額(元)/停業日期(迄)/董監事名單/舊營業項目資料/停業日期(起)/登記機關/最後核准變更日期/公司狀況/id/公司名稱


## 存入SQL

In [5]:
import sys
sys.getdefaultencoding()

'ascii'

In [6]:
import pypyodbc
import datetime

## 整理table
### 公司法人資料

In [7]:
datetime.datetime.strptime('20161201','%Y%m%d')

datetime.datetime(2016, 12, 1, 0, 0)

In [8]:

def get_company_info_str(data):
    if data:
        temp_info = ''
        for e in data:
            for j in e:                
                temp_info += '/'+j.encode('utf8') 
        return temp_info
    else:
        return None
print get_company_info_str(data0[u'所營事業資料'])
print get_company_info_str(data1[u'所營事業資料'])   


None
//機械開發工程之承攬//道路土木建築橋樑等之工程之承攬//土地開發規劃改良設計//社區國民住宅研究及設計//與前各項有關進築材料經銷及產製//有關前項業務之經營與投資


In [9]:
def get_date(data_date):
    if data_date:
        [month,day,year] = data_date.values()
        return datetime.datetime.strptime(str(year)+'-' + str(month)+'-' +str(day),'%Y-%m-%d')
    else:
        return None
# data0[u'停業日期(起)']

In [10]:
get_date(data0[u'停業日期(迄)'])

In [65]:
data_temp = {}
data_temp['id'] = str(data0['id'])
data_temp[u'公司狀況'] = data0[u'公司狀況']
data_temp[u'公司名稱'] = data0[u'公司名稱']
data_temp[u'資本總額(元)'] = int((data0[u'資本總額(元)']).replace(',',''))
data_temp[u'代表人姓名'] = data0[u'代表人姓名']
data_temp[u'公司所在地'] = data0[u'公司所在地']
data_temp[u'登記機關'] = data0[u'登記機關']
data_temp[u'核准設立日期'] = get_date(data0[u'核准設立日期'])
data_temp[u'最後核准變更日期'] = get_date(data0[u'最後核准變更日期'])
data_temp[u'所營事業資料'] = get_company_info_str(data0[u'所營事業資料'])
data_temp[u'停業日期(起)'] = get_date(data0[u'停業日期(起)'])
data_temp[u'停業日期(迄)'] = get_date(data0[u'停業日期(迄)'])
data_temp[u'舊營業項目資料'] = data0[u'舊營業項目資料']

In [70]:
data_temp.values()
datasets = [data_temp]

In [85]:
conn = pypyodbc.connect("DRIVER={SQL Server};SERVER=dbm_public;UID=sa;PWD=01060728;DATABASE=External;")
cursor = conn.cursor()

In [34]:
# def create_db(table,cursor):
sql_create = u"""
create table [公司法人資料]
(
    [ID] varchar(40) not null,
    [公司狀況] VARCHAR(10),
    [公司名稱] VARCHAR(100),
    [資本總額(元)] BIGINT,
    [代表人姓名] VARCHAR(50) ,
    [公司所在地] VARCHAR(200),
    [登記機關] VARCHAR(20),
    [核准設立日期] smalldatetime,
    [最後核准變更日期] smalldatetime,
    [所營事業資料] VARCHAR(3000),
    [停業日期(起)] smalldatetime,
    [停業日期(迄)] smalldatetime,
    [舊營業項目資料] VARCHAR(2000),
    constraint pk_id primary key (ID)
)
"""
cursor.execute(sql_create)
cursor.commit()

In [72]:
print fields

[代表人姓名],[所營事業資料],[核准設立日期],[公司所在地],[資本總額],[停業日期起],[舊營業項目資料],[登記機關],[停業日期迄],[最後核准變更日期],[公司狀況],[id],[公司名稱]


In [86]:

for index, row in enumerate(datasets):
    # row data
    # {key:value,key2:value2,...}
    fields = ','.join([ '['+e+ ']' for e in row.keys()])
    values = row.values()
    num_quest = '?' + ',?' * (len(row.keys())-1)
#     try:
    sql_insert = u"""INSERT INTO {0}({1}) VALUES({2})""".format(
        table, fields, num_quest)

    # print sql_insert ## check SQL SYNTAX
    cursor.execute(sql_insert, values)
    cursor.commit()
#     except:
#         print('error!!')

In [52]:
sql_insert = u"""INSERT INTO {0}({1}) VALUES({2})""".format(
        table, fields, num_quest)
print sql_insert

INSERT INTO 公司法人資料([u'[\u4ee3\u8868\u4eba\u59d3\u540d]', u'[\u6240\u71df\u4e8b\u696d\u8cc7\u6599]', u'[\u6838\u51c6\u8a2d\u7acb\u65e5\u671f]', u'[\u516c\u53f8\u6240\u5728\u5730]', u'[\u8cc7\u672c\u7e3d\u984d]', u'[\u505c\u696d\u65e5\u671f\u8d77]', u'[\u820a\u71df\u696d\u9805\u76ee\u8cc7\u6599]', u'[\u767b\u8a18\u6a5f\u95dc]', u'[\u505c\u696d\u65e5\u671f\u8fc4]', u'[\u6700\u5f8c\u6838\u51c6\u8b8a\u66f4\u65e5\u671f]', u'[\u516c\u53f8\u72c0\u6cc1]', '[id]', u'[\u516c\u53f8\u540d\u7a31]']) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?)


In [178]:
def to_db(datasets,table,cursor):
    
    for index, row in enumerate(datasets):
        # row data        
        fields = ','.join([ '['+e+ ']' for e in row.keys()]) # 欄位        
        values = row.values() # 值
        num_quest = '?' + ',?' * (len(row.keys())-1)
        try:
            sql_insert = u"""INSERT INTO {0}({1}) VALUES({2})""".format(
                table, fields, num_quest)
            
            cursor.execute(sql_insert, values)
            # print "row index : %d, success!" %index
        except pypyodbc.IntegrityError as e:
            print 'Data key Duplicate!! Forbiden'+str(e)    
    cursor.commit()

In [97]:
table = u'公司法人資料' 
to_db(datasets,table,cursor)

###  公司董監事資料

In [109]:
print '/'.join(data1[u'董監事名單'][0].keys())

序號/出資額/姓名/職稱/所代表法人


In [117]:
for e in data1[u'董監事名單']:
    print e

{u'\u5e8f\u865f': u'0000', u'\u51fa\u8cc7\u984d': u'1,000,000', u'\u59d3\u540d': u'\u738b\u632f\u6797', u'\u8077\u7a31': u'\u57f7\u884c\u696d\u52d9\u80a1\u6771', u'\u6240\u4ee3\u8868\u6cd5\u4eba': u''}


In [120]:
print '/'.join(data1[u'董監事名單'][0].keys())

序號/出資額/姓名/職稱/所代表法人


In [183]:
def get_director_lists(row):
    director_lists = []
    director = {}
    id = row['id']
    for e in row[u'董監事名單']:
        director['id'] = id
        director[u'姓名'] = e.get(u'姓名')
        director[u'職稱'] = e.get(u'職稱')
        director[u'出資額'] = int(e.get(u'出資額').replace(',',''))
        director[u'所代表法人'] = e.get(u'所代表法人')
        director_lists.append(director)
    return director_lists

In [184]:
to_db(get_director_lists(data1),u'公司董監事',cursor)

### 公司經理人資料

In [140]:
data3 = json.loads(data[100])

In [142]:
print '/'.join(data3[u'經理人名單'][0].keys())

序號/到職日期/姓名


In [144]:
data3[u'經理人名單'][0].values()

[u'0001', {u'day': 25, u'month': 10, u'year': 2011}, u'\u6797\u6b63\u5e38']

In [147]:
def get_manager_lists(row):
    manager_lists = []
    manager = {}
    id = row['id']
    for e in row[u'經理人名單']:
        manager['id'] = id
        manager[u'姓名'] = e.get(u'姓名')
        manager[u'到職日期'] = get_date(e.get(u'到職日期'))
        manager_lists.append(manager)
    return manager_lists

In [179]:
table = u'公司經理人'
data_manager_lists = get_manager_lists(data3)
to_db(data_manager_lists,table,cursor2)